In [189]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
import math
import pickle
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)
np.random.seed(123)
#https://www.analyticsvidhya.com/blog/2019/01/guide-pytorch-neural-networks-case-studies/
from sklearn.metrics import accuracy_score
from sklearn import linear_model, metrics, model_selection, ensemble
from sklearn.neural_network import MLPClassifier


In [238]:
predict_play = pickle.load(open('models/predict_play.sav', 'rb'))
rush_reg = pickle.load(open('models/rush_reg.sav', 'rb'))
pass_reg = pickle.load(open('models/pass_reg.sav', 'rb'))
fourth_down_classifier = pickle.load(open('models/4th_down_classifier.sav', 'rb'))
field_goal_classifier = pickle.load(open('models/field_goal_made.sav', 'rb'))

In [191]:
# just a df to play with
df = pd.read_csv('./data/PBP - 2016 - Week 1.csv')
special_teams_stats = pd.read_csv('./data/Special Teams Stats 2016.csv')
special_teams_stats = special_teams_stats.drop(columns=['Rk'])
offense_stats = pd.read_csv('./data/Offense Stats 2016.csv')
fulldf = df.join(special_teams_stats.set_index('School'), on='homeTeam')
fulldf = fulldf.join(offense_stats.set_index('School'), on='homeTeam')
fulldf = fulldf.dropna()
fulldf = fulldf.query('homeTeam == offenseTeam')
fulldf

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,FG made,FG attempted,FG%,Pts kicking,Punts,Punting yds,average yards per punt,kickoff return yards,kickoff return yards per kickoff,punt returns,punt return yards,punt return yards per return,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
0,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,15:00,2016-09-03T06:36:41Z,Kickoff,1,10,35,10,86,"Drew Galitz kickoff for 61 yds , Shakeir Ryan ...",0.8,1.2,62.5,6.8,4.4,181.5,41.4,51.5,19.1,1.8,17.2,9.7,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
6,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,0,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Pass Reception,1,10,41,6,47,Seth Russell pass complete to KD Cannon for 6 ...,0.8,1.2,62.5,6.8,4.4,181.5,41.4,51.5,19.1,1.8,17.2,9.7,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
7,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,1,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Rush,2,4,47,3,50,Shock Linwood run for 3 yds to the 50 yard line,0.8,1.2,62.5,6.8,4.4,181.5,41.4,51.5,19.1,1.8,17.2,9.7,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
8,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,2,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Penalty,3,1,50,-10,40,"Linwood,Shock rush for 3 yards to the NWST47 (...",0.8,1.2,62.5,6.8,4.4,181.5,41.4,51.5,19.1,1.8,17.2,9.7,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
9,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,1,3,239,Baylor,BAY,2466,Northwestern St,NWST,0,0,1,12:45,2016-09-03T06:41:38Z,Rush,3,11,40,42,82,Shock Linwood run for 42 yds to the NWSt 18 fo...,0.8,1.2,62.5,6.8,4.4,181.5,41.4,51.5,19.1,1.8,17.2,9.7,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16136,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,25,0,2653,Troy,TROY,2046,Austin Peay,PEAY,57,17,4,3:18,2016-09-04T08:15:02Z,Rush,1,10,33,15,48,B.J. Smith run for 15 yds to the Troy 48 for a...,1.4,1.8,78.3,8.0,5.3,216.9,40.9,63.9,22.5,1.8,15.3,8.3,39.0,13.0,33.7,24.2,37.8,64.0,260.5,1.9,37.3,169.2,4.5,2.2,75.2,429.6,5.7,11.8,9.2,6.8,61.8
16137,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,25,1,2653,Troy,TROY,2046,Austin Peay,PEAY,57,17,4,3:18,2016-09-04T08:15:02Z,Rush,1,10,48,3,51,B.J. Smith run for 3 yds to the APeay 49,1.4,1.8,78.3,8.0,5.3,216.9,40.9,63.9,22.5,1.8,15.3,8.3,39.0,13.0,33.7,24.2,37.8,64.0,260.5,1.9,37.3,169.2,4.5,2.2,75.2,429.6,5.7,11.8,9.2,6.8,61.8
16138,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,25,2,2653,Troy,TROY,2046,Austin Peay,PEAY,57,17,4,3:18,2016-09-04T08:15:02Z,Rush,2,7,51,-2,49,B.J. Smith run for a loss of 2 yards to the Tr...,1.4,1.8,78.3,8.0,5.3,216.9,40.9,63.9,22.5,1.8,15.3,8.3,39.0,13.0,33.7,24.2,37.8,64.0,260.5,1.9,37.3,169.2,4.5,2.2,75.2,429.6,5.7,11.8,9.2,6.8,61.8
16139,400869810,2016,1,2653,Troy,TROY,2046,Austin Peay,PEAY,25,3,2653,Troy,TROY,2046,Austin Peay,PEAY,57,17,4,3:18,2016-09-04T08:15:02Z,Rush,3,9,49,8,57,B.J. Smith run for 8 yds to the A

In [192]:
def seconds_left_in_quarter(clock):
    clock = clock.split(':')
    clock = (int(clock[0])*60) + int(clock[1])
    return clock

In [ ]:
for index,row in fulldf.iterrows():
    fulldf.at[index,'clock'] = seconds_left_in_quarter(row['clock'])

In [103]:
sample_row = fulldf[35:36]
sample_row

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,FG made,FG attempted,FG%,Pts kicking,Punts,Punting yds,average yards per punt,kickoff return yards,kickoff return yards per kickoff,punt returns,punt return yards,punt return yards per return,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
53,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,8,0,239,Baylor,BAY,2466,Northwestern St,NWST,24,0,1,62,2016-09-03T07:28:14Z,Kickoff,1,0,35,0,90,"Drew Galitz kickoff for 65 yds , Shakeir Ryan ...",0.8,1.2,62.5,6.8,4.4,181.5,41.4,51.5,19.1,1.8,17.2,9.7,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2


In [94]:
def get_down(row):
    return int(row['down'])

In [210]:
# check if 4th down
def play_and_result(sample_row):
    if(get_down(sample_row) == 4):
        # if it's 4th down use the fourth_down_classifier to determine what it will do

        # if it's a fieldgoal use the field_goal classifier to determine result
        pass
    else:
        print('not 4th')
        feature = sample_row[['quarter', 'down', 'distance', 'yardLine', 'clock']]
        feature = feature.values.reshape(1, -1)
        play = predict_play.predict(feature)[0]
        print(play)
        if(play == 'Rush'):
            feature = sample_row[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts',
                                'Rushing yards', 'rush average', 'Rush Touchdowns',
           'Total Offense Plays', 'Total yards', 'total yards per play',
           'First down by pass', 'First down by rush']]
            feature = feature.values.reshape(1,-1)
            yards_gained = rush_reg.predict(feature)
            return get_down(sample_row), yards_gained[0], sample_row['distance']
        else:
            feature = sample_row[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps',
       'completion percentage', 'passing yards', 'Passing Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]
            feature = feature.values.reshape(1,-1)
            yards_gained = pass_reg.predict(feature)
            return get_down(sample_row), yards_gained[0], sample_row['distance']
    # if it's not 4th down take the position and put it into predict_play

    # if the result is a run use the rush_reg, else use the pass_reg

    # get the result of that and update the field position, increment the down if necessary

    # repeat until score or 4th down

In [118]:
play_and_result(sample_row)

not 4th
Rush


(1, 6.059528251063969, 10)

In [202]:
def update_position(sample_row, old_down, calc_distance, old_distance):
    new_row = sample_row
    new_distance = old_distance - calc_distance
    new_yard_line = int(new_row['yardLine']) + calc_distance
    if(new_distance < 0):
        new_row['down'] = 1
        # need to check if it's close to the goal line :/
        new_row['distance'] = 10
        new_row['yardLine'] = new_yard_line
    else:
        old_down+=1
        new_row['distance'] = new_distance
        new_row['down'] = old_down
        new_row['yardLine'] = new_yard_line
    return new_row

In [120]:
old_down, calc_distance, old_distance = play_and_result(sample_row)
new_row = update_position(sample_row, old_down, calc_distance, old_distance)

not 4th
Rush


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [217]:
plays = []
for index,row in fulldf.iterrows():
    new_row = row
    down = get_down(new_row)
    i = 0
    while (new_row['yardLine'] < 100 and down != 4):
        i+=1
        print(i)
        old_down, calc_distance, old_distance = play_and_result(new_row)
        new_row = update_position(new_row, old_down, calc_distance, old_distance)
        down = get_down(new_row)
    plays.append(i)

1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
1
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
1
not 4th
Pass
2
not 4th
Rush
3
not 4th
Rush
1
not 4th

18
not 4th
Rush
1
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Rush
2
not 4th
Pass
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
17
not 4th
Rus

Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
17
not 4th
Rush
1
not 4th
Rush
2
not 4th
Pass
3
not 4th
Pass
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
17
not 4th
Rush
1
not 4th
Pass
2
not 4th
Pass
1
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
1
not 4th
Rush
2
not 4th
Pass
3
not 4th
Pass
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4t

Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
17
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th


KeyboardInterrupt: 

In [234]:
def fourth_down_play(row):
    features = row[['homeScore', 'awayScore', 'quarter', 'distance', 
                    'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps', 'completion percentage', 
                    'passing yards', 'Passing Touchdowns', 'Rush Attempts', 'Rushing yards', 'rush average', 
                    'Rush Touchdowns', 'Total Offense Plays', 'Total yards', 'total yards per play', 
                    'First down by pass', 'First down by rush']].values.reshape(1, -1)
    result = fourth_down_classifier.predict(features)[0]
    #test_labels = test_labels.replace('Punt', 0)
    # test_labels = test_labels.replace('Field Goal', 1)
    # test_labels = test_labels.replace('Pass', 2)
    # test_labels = test_labels.replace('Rush', 3)
    if(result == 0):
        return 'Punt'
    elif result ==1:
        return 'Field Goal'
    elif result == 2:
        return 'Pass'
    else:
        return 'Rush'

In [241]:
def field_goal_result(row):
    features = row[['yardLine', 'FG made', 'FG%']].values.reshape(1,-1)
    result = field_goal_classifier.predict(features)[0]
#         if(df_type == 'Field Goal Good'):
#         return 1
#     else:
#         return 0
    if(result == 1):
        return 'Field Goal Good'
    else:
        return 'Missed'

In [242]:
field_goal_result(test_row)

'Missed'

In [263]:
def fourth_down(test_row):
    play = fourth_down_play(test_row)
    if play == 'Field Goal':
        result = field_goal_result(test_row)
        if(result == 'Field Goal Good'):
            # add 3 points
            return 3
        else:
            return 'break'
    elif play == 'Pass':
        feature = test_row[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts',
                            'Passing Completion', 'Pass Attemps',
                            'completion percentage', 'passing yards', 'Passing Touchdowns',
                            'Total Offense Plays', 'Total yards', 'total yards per play',
                            'First down by pass', 'First down by rush']].values.reshape(1,-1)
        yards_gained = pass_reg.predict(feature)
        if(test_row['yardLine'] + yards_gained >= 100):
            return 7
        elif(yards_gained >= test_row['distance']):
            return yards_gained
        else:
            return 'break'
    elif play == 'Rush':
        feature = test_row[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts',
                            'Rushing yards', 'rush average', 'Rush Touchdowns',
                            'Total Offense Plays', 'Total yards', 'total yards per play',
                            'First down by pass', 'First down by rush']].values.reshape(1,-1)
        yards_gained = rush_reg.predict(feature)
        if(test_row['yardLine'] + yards_gained >= 100):
            return 7
        elif(yards_gained >= test_row['distance']):
            return yards_gained
        else:
            return 'break'


In [264]:
fourth_down(test_row)

'break'

In [270]:
# check if 4th down
def play_and_result(sample_row):
    if(get_down(sample_row) == 4):
        result = fourth_down(sample_row)
        if (result ==3 | result ==7 | result == 'break'):
            return result
        else:
            return 0, result, sample_row['distance']
    else:
        feature = sample_row[['quarter', 'down', 'distance', 'yardLine', 'clock']]
        feature = feature.values.reshape(1, -1)
        play = predict_play.predict(feature)[0]
        print(play)
        if(play == 'Rush'):
            feature = sample_row[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts',
                                'Rushing yards', 'rush average', 'Rush Touchdowns',
           'Total Offense Plays', 'Total yards', 'total yards per play',
           'First down by pass', 'First down by rush']]
            feature = feature.values.reshape(1,-1)
            yards_gained = rush_reg.predict(feature)
            return get_down(sample_row), yards_gained[0], sample_row['distance']
        else:
            feature = sample_row[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps',
       'completion percentage', 'passing yards', 'Passing Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]
            feature = feature.values.reshape(1,-1)
            yards_gained = pass_reg.predict(feature)
            return get_down(sample_row), yards_gained[0], sample_row['distance']
    # if it's not 4th down take the position and put it into predict_play

    # if the result is a run use the rush_reg, else use the pass_reg

    # get the result of that and update the field position, increment the down if necessary

    # repeat until score or 4th down

In [273]:
result = play_and_result(test_row)

Rush


(1, 5.040928217403135, 10)

In [286]:
old_down, calc_distance, old_distance = play_and_result(test_row)
new_row = update_position(new_row, old_down, calc_distance, old_distance)

Rush


In [287]:
old_down, calc_distance, old_distance = play_and_result(new_row)
new_row = update_position(new_row, old_down, calc_distance, old_distance)
new_row

Rush


gameId                                                                      400868910
year                                                                             2016
week                                                                                1
homeId                                                                            218
homeTeam                                                                       Temple
homeAbbr                                                                          TEM
awayId                                                                            349
awayTeam                                                                         Army
awayAbbr                                                                         ARMY
driveIndex                                                                          4
playIndex                                                                           1
offenseId                                             